In [1]:
import pandas as pd
import numpy as np
import scipy 

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from random import choices,sample

import os
import gc
import pickle

import warnings
warnings.filterwarnings('ignore')

from datetime import timedelta
from tqdm import tqdm

plt.style.use('ggplot')

mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = "Malgun Gothic"

os.chdir("../")
root_path = os.getcwd()

data_folder_path = os.path.join(root_path, 'data')
original_file_path = os.path.join(data_folder_path, 'original_data')
original_raw_file_path = os.path.join(original_file_path,'raw_data')
original_processed_file_path = os.path.join(original_file_path,'processed_data')

raw_file_folders = os.listdir(original_raw_file_path)

external_file_path = os.path.join(data_folder_path,'external_data')
external_raw_file_path = os.path.join(external_file_path,'raw_data')
external_processed_file_path = os.path.join(external_file_path,'processed_data')

c:\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
seoul_pop = pd.read_table(os.path.join(external_raw_file_path,"행정동_가구원수별_가구수.txt"), header=[1], thousands=',')
daegu_pop = pd.read_csv(os.path.join(external_raw_file_path,"구·군_동·읍·면별_세대원수별_세대수_20200809161016.csv"), encoding='cp949', header=[1])

seoul_pop.drop('기간', axis=1, inplace=True)
daegu_pop = daegu_pop.iloc[:,[0,1,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]]

In [4]:
cj = pd.read_csv(os.path.join(original_processed_file_path, "CJ_delivery.csv"), thousands=',')
cj_cat_merge = cj.groupby(['DL_YMD','CTPV_NM','CTGG_NM','HDNG_NM']).sum().iloc[:,-1].reset_index(name='INVC_CONT')

In [7]:
daegu_pop.columns = ['GU_NM','HDONG_NM','total','pop1','pop2','pop3','pop4','pop5','pop6','pop7','pop8','pop9']
seoul_pop.columns = ['GU_NM','HDONG_NM','total','pop1','pop2','pop3','pop4','pop5','pop6','pop7']

In [8]:
total_dong = cj.HDNG_NM.unique().tolist()

total_gu = ['중구','수성구','노원구']
total_dong += ['수성2·3가동']

In [9]:
daegu_pop = daegu_pop.loc[(daegu_pop.HDONG_NM.isin(total_dong)) & (daegu_pop.GU_NM.isin(total_gu))].reset_index(drop=True)
seoul_pop = seoul_pop.loc[(seoul_pop.HDONG_NM.isin(total_dong)) & (seoul_pop.GU_NM.isin(total_gu))].reset_index(drop=True)

In [10]:
cj_cat_merge.rename(columns={'CTGG_NM':'GU_NM', 'HDNG_NM':'HDONG_NM'}, inplace=True)

In [11]:
cj_cat_merge=cj_cat_merge.replace('수성2·3가동', '수성2.3가동')
cj_cat_merge=cj_cat_merge.replace('신당제5동', '신당5동')

In [25]:
"""
cj_cat_merge['DL_YMD'] = pd.to_datetime(cj_cat_merge['DL_YMD'].apply(lambda x:'20'+str(x)), format='%Y%m%d')

cj_cat_merge['dayofweek'] = cj_cat_merge.DL_YMD.dt.dayofweek.values
cj_cat_merge['weekday'] = np.where(cj_cat_merge.dayofweek < 5, '평일', '주말')

cj_cat_merge = cj_cat_merge.loc[cj_cat_merge.weekday=='평일'].reset_index()
"""

In [12]:
daegu_pop=daegu_pop.replace('수성2·3가동', '수성2.3가동')

In [13]:
tmp1=cj_cat_merge.merge(daegu_pop)
tmp2=cj_cat_merge.merge(seoul_pop)

In [14]:
tmp1=tmp1.iloc[:,:-2]

In [15]:
cj_with_pop = pd.concat([tmp1,tmp2]).reset_index(drop=True)

In [16]:
cj_with_pop.iloc[:,4:].corr().iloc[:1,2:]

,pop1,pop2,pop3,pop4,pop5
INVC_CONT,0.350729,0.377958,0.394308,0.351095,0.339797


주말포함

In [17]:
cj_with_pop.iloc[:,4:].corr().iloc[:1,3:]

,pop2,pop3,pop4,pop5
INVC_CONT,0.377958,0.394308,0.351095,0.339797


주말제외